In [1]:
from alfabeto_sources.all_sources import *
from alfabeto_data.pickled_data import *
from Continuo.kmeans_inversion import k_means_data, inversion_data, silhouette_analysis, label_maker_alf
from alfabeto_data import hmm

import numpy as np
from sklearn import metrics
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

from music21 import *
import time, pickle

In [2]:
def crd_data(unparsed):  
    def parser(song):
        x = converter.parse(song)
        if type(x) is stream.Score:
            crd = x.chordify().flat.getElementsByClass(chord.Chord)
            key_sig = x.flat.getElementsByClass(key.KeySignature)[0].sharps
            bass_note = crd[-1].bass().pitchClass
            chordified.append(crd)
            all_keys.append((key_sig, bass_note))
        elif type(x) is stream.Opus:
            for y in x:
                crd = y.chordify().flat.getElementsByClass(chord.Chord)
                key_sig = y.flat.getElementsByClass(key.KeySignature)[0].sharps
                bass_note = crd[-1].bass().pitchClass
                chordified.append(crd)
                all_keys.append((key_sig, bass_note))
    start = time.time()
    for x in unparsed:
        st = time.time()
        parser(x)
        fin = time.time()
        print('finished', len(chordified), 'of', len(unparsed), 'in', fin - st, 'seconds')
    print('entire process took:', time.time() - start, 'seconds')

    bach_chords_pickle_out = open("pickles/bach_chords.pickle", "wb")
    pickle.dump((chordified, all_keys), bach_chords_pickle_out)
    bach_chords_pickle_out.close()

In [3]:
A = corpus.getComposer('bach')
all_parsed = []
all_keys = []
chordified = []
crd_data(A)

finished 1 of 426 in 12.317260503768921 seconds
finished 2 of 426 in 5.0271155834198 seconds
finished 3 of 426 in 3.6107840538024902 seconds
finished 4 of 426 in 4.334396600723267 seconds
finished 5 of 426 in 6.450267553329468 seconds
finished 6 of 426 in 6.15505051612854 seconds
finished 7 of 426 in 5.614304304122925 seconds
finished 8 of 426 in 10.261813402175903 seconds
finished 9 of 426 in 4.05670952796936 seconds
finished 10 of 426 in 7.919573545455933 seconds
finished 11 of 426 in 10.331655263900757 seconds
finished 12 of 426 in 6.02640438079834 seconds
finished 13 of 426 in 5.802277088165283 seconds
finished 14 of 426 in 5.923980474472046 seconds
finished 15 of 426 in 5.102010488510132 seconds
finished 16 of 426 in 4.141759395599365 seconds
finished 17 of 426 in 6.59498143196106 seconds
finished 18 of 426 in 4.789719343185425 seconds
finished 19 of 426 in 7.657521486282349 seconds
finished 20 of 426 in 6.512287855148315 seconds
finished 21 of 426 in 16.98009419441223 seconds
fin

In [ ]:
def k_means_simple(list_of_lists, cluster_number, label_markers):
    np.random.seed(42)

    data = np.array(list_of_lists)
    label_dict = {}
    label_numbers = []
    n = 0
    label_dict[label_markers[0]] = 0
    for j in range(1, len(label_markers)):
        if label_markers[j] not in label_dict:
            label_dict[label_markers[j]] = n+1
            n += 1

    for i in label_markers:
        label_numbers.append(label_dict[i])
    n_samples, n_features = data.shape
    n_digits = cluster_number
    labels = np.array(label_numbers)
    # print('label numbers', label_numbers)
    # sample_size = 300

    sample_size = len(data)    

    k_data = KMeans(init='k-means++', n_clusters=n_digits, n_init=10).fit(data)
    silhouette = metrics.silhouette_score(data, k_data.labels_, metric='euclidean', sample_size=sample_size)
    completeness = metrics.completeness_score(labels, k_data.labels_)
    return silhouette, completeness

def label_maker(label_tuples):
    final_labels = []
    for x in label_tuples:
        final_labels.append('$'+str(x[0])+':'+str(x[1])+'$')
    return final_labels

In [ ]:
for x, y in chronological.items():
    print(x, k_means_simple(inversion_data(y), 2, label_maker_alf(y)))

In [ ]:
print(k_means_simple(liber_usalis_notes_data[0], 6, label_maker(liber_usalis_notes_data[1])))

print(k_means_simple(josquin_notes_data[0], 6, label_maker(josquin_notes_data[1])))
print(k_means_simple(palestrina_notes_data[0], 6, label_maker(palestrina_notes_data[1])))
k_means_simple(bach_notes_data[0], 6, label_maker(bach_notes_data[1]))

In [ ]:
k_means_simple(inversion_data(chronological[1620]), 6, 
               label_maker_alf(chronological[1620]))

In [ ]:
def double_figure(list_of_lists, list_of_labels, max_clusters):
    
    cluster_number = []
    silhouette = []
    completeness = []
    for x in range(2, max_clusters+1):
        cluster_number.append(x)
        silhouette.append(k_means_simple(list_of_lists, x, list_of_labels)[0]) 
        completeness.append(k_means_simple(list_of_lists, x, list_of_labels)[1])
    
        # Create a subplot with 1 row and 2 columns
    plt.scatter(cluster_number, silhouette, color='blue', label='silhouette')
    plt.scatter(cluster_number, completeness, color='green', label='completeness')
#     fig, (ax1, ax2) = plt.subplots(1, 2)
#     fig.set_size_inches(18, 7)

    # The 1st subplot is the cluster plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
#     ax1.set_xlim([-0.1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
#     ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])
#     ax1.set_title("PCA-Reduced K-Means Graph")
#     ax1.set_xlabel("The silhouette coefficient values")
#     ax1.set_ylabel("Cluster label")


#     # 2nd Plot showing the actual clusters formed
#     colors = cm.spectral(cluster_labels.astype(float) / n_clusters)
#     ax2.scatter(X[:, 0], X[:, 1], marker='.', s=30, lw=0, alpha=0.7,
#                 c=colors)

#     # Labeling the clusters
#     centers = clusterer.cluster_centers_
#     # Draw white circles at cluster centers
#     ax2.scatter(centers[:, 0], centers[:, 1],
#                 marker='o', c="white", alpha=1, s=200)

#     for i, c in enumerate(centers):
#         ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1, s=50)

#     ax2.set_title("The visualization of the clustered data.")
#     ax2.set_xlabel("Feature space for the 1st feature")
#     ax2.set_ylabel("Feature space for the 2nd feature")

#     plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
#                   "with n_clusters = %d" % n_clusters),
#                  fontsize=14, fontweight='bold')
    plt.xlabel('Number of Clusters')
    plt.ylabel('Score')
    plt.legend()
    plt.show()

In [ ]:

double_figure(trecento_notes_data[0], label_maker(trecento_notes_data[1]), 12)

double_figure(josquin_notes_data[0], label_maker(josquin_notes_data[1]), 12)

double_figure(palestrina_notes_data[0], label_maker(palestrina_notes_data[1]), 12)

double_figure(alfabeto_notes_data[0], label_maker(alfabeto_notes_data[1]), 12)

double_figure(bach_notes_data[0], label_maker(bach_notes_data[1]), 12)

double_figure(corelli_notes_data[0], label_maker(corelli_notes_data[1]), 12)